In [1]:
from flask import Flask, request, jsonify
from typing import Dict, List, Tuple, Any, Annotated, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, AnyMessage, SystemMessage
from langchain_core.tools import BaseTool
from langgraph.graph import START, StateGraph, END
from langchain_core.agents import AgentAction, AgentFinish
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver


In [2]:
from research_agent import ResearchAgent, llm

In [3]:
research_agent = ResearchAgent(llm)

In [4]:
human_task = "plan a travel plan for exploring athens then traveling to kefalonia and exploring it for 3 days, must be accurate from information directly or inderictle from the web, of the time taken to travel between places"

In [5]:
response = research_agent.graph.invoke({"messages": [HumanMessage(content=human_task)]})

planning node response: content='1. **Research Athens Exploration**  \n   1.1. Identify key attractions in Athens (e.g., Acropolis, Parthenon, Plaka).  \n   1.2. Determine the best transportation options within Athens (e.g., metro, bus, walking).  \n   1.3. Estimate the time required to visit each attraction.\n\n2. **Plan Travel from Athens to Kefalonia**  \n   2.1. Research transportation options from Athens to Kefalonia (e.g., ferry, flight).  \n   2.2. Check the schedule and duration of the chosen transportation method.  \n   2.3. Book tickets for the journey to Kefalonia.\n\n3. **Research Kefalonia Exploration**  \n   3.1. Identify key attractions in Kefalonia (e.g., Myrtos Beach, Melissani Cave, Assos).  \n   3.2. Determine the best transportation options within Kefalonia (e.g., rental car, bus).  \n   3.3. Create a daily itinerary for the 3 days of exploration, including estimated travel times between attractions.' additional_kwargs={'refusal': None} response_metadata={'token_usa

In [5]:
response = research_agent.graph.invoke({"messages": [HumanMessage(content="human_task")], "task_list": "", "original_task": "human_task"})

planning node prompt: [SystemMessage(content="\n            You are a task breaking down assistant. Your job is to make step or steps, from the current task into smaller tasks and steps for each task.\n\n            For example, if someone asks about interesting places to visit in two different cities, break it down into two steps for each city.\n            If they ask about a single location, create just one step.\n\n            Important rules:\n            - Always number your steps, even if there's only one\n            - Don't execute the tasks yourself\n            - Don't ask the user for clarification\n            - There should not be redundant steps\n            - Try to always make no more than 3 steps per smaller task, unless you really really need to\n            \n\n            This is the current task to be deconstructed:\n            Give me list of cities to cityhopping from algarve to lisboa\n            ", additional_kwargs={}, response_metadata={})]
planning node r

In [5]:
print(response)

{'messages': [HumanMessage(content='Give me list of places to visit in Rome and Tuscany', additional_kwargs={}, response_metadata={}, id='b0b1075d-049f-4722-8287-f55f6f43518d'), AIMessage(content="1. Identify interesting places to visit in Rome.\n   1.1. Research historical landmarks (e.g., Colosseum, Roman Forum).\n   1.2. Look for cultural attractions (e.g., museums, art galleries).\n   1.3. Find popular neighborhoods and parks (e.g., Trastevere, Villa Borghese).\n\n2. Identify interesting places to visit in Tuscany.\n   2.1. Research famous cities (e.g., Florence, Siena).\n   2.2. Look for scenic landscapes and vineyards (e.g., Chianti region).\n   2.3. Find historical sites and landmarks (e.g., Pisa's Leaning Tower, San Gimignano).", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 146, 'prompt_tokens': 172, 'total_tokens': 318, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejec

In [1]:
from research_agent import tavily_tool

In [2]:
tavily_tool.invoke("What is the capital of Italy?")

{'query': 'What is the capital of Italy?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'What Is the Capital of Italy? - WorldAtlas',
   'url': 'https://www.worldatlas.com/articles/what-is-the-capital-of-italy.html',
   'content': 'Rome is the largest and the capital of Italy, as well as the capital of the Lazio region. The city has a population of about 2.9 million and its metropolitan area has population of 4.3 million, making it the fourth most populous city in the European Union (EU). The Vatican is an independent state located entirely within the city of Rome, and remains the only country within a city. Rome is a global city and a popular tourist destination. Economy As the capital of Italy, Rome is home to various government institutions including the presidency, legislature, judiciary, and diplomatic representatives.',
   'score': 0.92995757,
   'raw_content': None},
  {'title': 'Rome - Wikipedia',
   'url': 'https://en.wikipedia.org/wiki/

finalizer node response: content="Here is a comprehensive list of places to visit in Mallorca, organized by categories:\n\n### Beaches\n1. **Es Trenc** - Famous for its white sand and turquoise waters.\n2. **Cala Millor** - A vibrant beach destination with clear waters.\n3. **Playa de Muro** - Family-friendly beach known for its amenities.\n4. **Cala Mesquida** - A natural area with crystal-clear water, ideal for relaxation.\n\n### Museums\n1. **Palma Aquarium** - A marine research center with diverse sea life.\n2. **Archaeological Museum** - Showcasing Mallorca's rich historical artifacts.\n3. **Museu Diocesà** - A museum dedicated to religious art and history.\n\n### Outdoor Activities\n1. **Hiking in Serra de Tramuntana** - Explore the UNESCO World Heritage Site with stunning views.\n2. **Caves of Drach Tour** - A guided tour through impressive underground caves.\n3. **Boat Tours** - Experience the coastline and hidden coves by boat.\n\n### Historical Sites\n1. **Catedral de Mallorca (La Seu)** - A Gothic cathedral in Palma.\n2. **Bellver Castle** - A unique circular fortress from the 14th century.\n3. **Arab Baths (Banys Arabs)** - Well-preserved baths from the Moorish period.\n4. **Santuari de Lluc** - A revered monastery in the Tramuntana mountains.\n5. **Capdepera Castle** - A hilltop fortress with stunning views.\n6. **Palace of la Almudaina** - A royal palace with historical significance.\n\n### Natural Landmarks\n1. **Serra de Tramuntana** - A UNESCO World Heritage Site known for its stunning landscapes.\n2. **Cuevas del Drach** - Impressive caves featuring underground lakes.\n\n### Cultural Experiences\n1. **Local Wine Tastings** - Experience Mallorca's wine culture through tastings.\n2. **Art Galleries and Museums** - Explore local art and history through various exhibitions.\n3. **Traditional Festivals** - Participate in local celebrations and cultural events.\n\nFor more information, you can visit the following links:\n- [Spain.info - Majorca](https://www.spain.info/gcc/en/destinations/majorca/)\n- [Lonely Planet - Attractions in Mallorca](https://www.lonelyplanet.com/spain/mallorca/attractions)\n- [TripAdvisor - Things to Do in Majorca](https://www.tripadvisor.com/Attractions-g187462-Activities-Majorca_Balearic_Islands.html)\n- [U.S. News Travel - Best Things to Do in Mallorca](https://travel.usnews.com/Mallorca_Spain/Things_To_Do/)\n- [View Mallorca - Historical Sites](https://www.viewmallorca.com/single-article-page-list/five-of-palmas-historical-sites-and-architectural-wonders)\n- [See Mallorca - Cultural Activities](https://www.seemallorca.com/cultural/guide)\n\nEnjoy your visit to Mallorca!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 624, 'prompt_tokens': 3991, 'total_tokens': 4615, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiJdKBlztpkQL1yO1Sc0s6zXkGBIt', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--b2c5eebe-a2e1-4830-967c-675746824bf2-0' usage_metadata={'input_tokens': 3991, 'output_tokens': 624, 'total_tokens': 4615, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

finalizer node response: content="Here is a list of notable cities for city hopping from Algarve to Lisboa, along with highlights for each city:\n\n1. **Évora**\n   - **Highlights**: A UNESCO World Heritage site known for its Roman Temple, Chapel of Bones, and medieval architecture. Évora's rich history and charming streets make it a must-visit.\n   - [Visit Évora](https://www.visitevora.net/en/visit-evora/)\n\n2. **Beja**\n   - **Highlights**: Known for its historical significance, Beja features a medieval castle, the Beja Regional Museum, and beautiful parks. It offers a peaceful atmosphere with traditional Alentejo life.\n   - [Explore Beja](https://www.beportugal.com/beja-portugal/)\n\n3. **Mértola**\n   - **Highlights**: A picturesque hilltop town with a rich Islamic heritage, featuring a well-preserved castle and stunning views of the Guadiana River. The Mértola Islamic Festival is a highlight.\n   - [Discover Mértola](https://portugaltravelguide.com/mertola/)\n\n4. **Monchique**\n   - **Highlights**: Nestled in the Serra de Monchique mountains, this town is known for its natural beauty, hiking trails, and thermal springs. It offers a peaceful retreat with stunning views.\n   - [Visit Monchique](https://wetravelportugal.com/monchique-portugal/)\n\n5. **Silves**\n   - **Highlights**: Once the capital of the Algarve, Silves is famous for its Moorish castle and Gothic cathedral. The town's historical significance is reflected in its architecture.\n   - [Explore Silves](https://www.algarve-tourist.com/silves-portugal-guide.html)\n\n6. **Loulé**\n   - **Highlights**: A vibrant market town known for its traditional gypsy market and Moorish influences. Loulé offers a mix of culture, history, and beautiful surroundings.\n   - [Discover Loulé](https://www.algarve-tourist.com/loule-portugal-algarve.html)\n\n7. **Lagoa**\n   - **Highlights**: Renowned for its stunning beaches, including Praia da Marinha, Lagoa is also known for its wine production and charming coastal scenery.\n   - [Visit Lagoa](https://www.algarveportugaltourism.com/lagoa/)\n\n8. **Portimão**\n   - **Highlights**: A bustling port city famous for its lively marina, beautiful beaches, and vibrant nightlife. Praia da Rocha is a popular beach destination.\n   - [Explore Portimão](https://www.algarve-tourist.com/portimao-portugal-algarve.html)\n\n9. **Sines**\n   - **Highlights**: Known as the birthplace of Vasco da Gama, Sines features a historic castle and beautiful beaches. The town has a rich maritime history.\n   - [Discover Sines](https://www.portugal.com/city/a-guide-to-sines-portugals-port-of-entry/)\n\n10. **Setúbal**\n    - **Highlights**: Located near the Sado River, Setúbal is known for its seafood, beautiful beaches, and the stunning Arrábida Natural Park.\n    - [Visit Setúbal](https://visitsetubal.com/en/)\n\n11. **Sesimbra**\n    - **Highlights**: A charming fishing town famous for its beautiful beaches, seafood restaurants, and the historic Sesimbra Castle.\n    - [Explore Sesimbra](https://www.beportugal.com/sesimbra/)\n\nFor more information on traveling between Algarve and Lisboa, you can check out these resources:\n- [Algarve to Lisbon with stops up to 3 cities](https://www.mywonderstours.com/tour/algarve-to-lisbon-with-stops-up-to-3-cities)\n- [Algarve to Lisbon: The Ultimate Travel Itinerary](https://theportugaldaily.com/visit/destinations/algarve-to-lisbon-the-ultimate-travel-itinerary)\n\nThis list provides a great starting point for your city hopping adventure in Portugal!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 860, 'prompt_tokens': 19603, 'total_tokens': 20463, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiKYmvlxWEQa7UHzEfKc3kwWJ24PQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--74d8c703-8abf-4c45-a3ee-768c14a7441c-0' usage_metadata={'input_tokens': 19603, 'output_tokens': 860, 'total_tokens': 20463, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}